In [ ]:
import pandas as pd
import numpy as np
import logging
import os


In [ ]:
def extract(a,b):
    b_df = pd.read_parquet(b)
    merged_df = a.merge(b_df, on = 'index')
    return merged_df

def transform(df):
    df.fillna(
        {
        'Weekly_Sales': df['Weekly_Sales'].mean(),
        'CPI': df['CPI'].mean(),
        'Unemployment': df['Unemployment'].mean(),
        }, inplace = True
    )
    
    df['Date'] = pd.to_datetime(df['Date'], format = "%Y-%m-%d")
    df['Month'] = df['Date'].dt.month
    
    over_10k_df = df.loc[df['Weekly_Sales'] >10000,:]
    
    df = over_10k_df.drop(["index", "Temperature", "Fuel_Price", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4",                         "MarkDown5", "Type", "Size", "Date"], axis = 1)
    return df
    
def avg_monthly_sales(df):
 
    agg_data = pd.DataFrame(df.groupby('Month')['Weekly_Sales'].mean().round(2)).reset_index().rename(columns ={'Weekly_Sales':'Avg_Sales'})
    return agg_data
    
def load(clean_data, clean_data_path, agg_data, agg_data_path):
    clean_data.to_csv('clean_data_path',index=False)
    agg_data.to_csv('agg_data_path',index=False)

def validation(file):
    if os.path.isfile(file) is not True:
        return 'File doesnt exist'
    else:
        return 'File exists'
    

In [ ]:
merged_df = extract(grocery_sales, 'extra_data.parquet')
clean_data = transform(merged_df)
agg_data = avg_monthly_sales(clean_data)
load(clean_data, 'clean_data.csv', agg_data, 'agg_data.csv')
validation('clean_data.csv')
validation('agg_data.csv')